 # Opis notatnika
 W poprzednich krokach pobraliśmy dane oraz przygotowaliśmy bazę `Postgres` na import. Głównym celem w tym notatniku jest  odpowiednie dostosowanie struktury danych z plików źródłowych do formatu zgodnego z `Postgres`, a następnie wgranie ich na nasz serwer. Dzięki temu w późniejszych krokach możemy niezależnie użyć danych do analizy czy raportowania.

 Przy wykonywaniu tego notebooka przydadzą się poniższe elementy kursu oraz materiały dodatkowe:
 * `SQL - analiza danych > Zjazd 1 - materiały dodatkowe > Export danych z DB > Python` - w celu użycia połączenia razem z `Pandas`,
 * https://docs.sqlalchemy.org/en/14/core/engines.html - w celu uzupełnienia konfiguracji `Pandas` do `PostgerSQL`,
 * https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html - eksport danych z `Pandas` na bazę danych.

 > Uwaga: Ze względu na wolumen danych zawarty w pliku `flight.csv`, wykonanie tego notatnika może zająć nawet kilkadziesiąt minut lub więcej!

Tutaj zaimportuj wymagane biblioteki

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sys
sys.path.append('../')

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [2]:
from config.DB_connect import username, passwd, hostname, db_name, port

 Z pomocą artykułu [click](https://docs.sqlalchemy.org/en/14/core/engines.html) stwórz zmienną `engine` oraz `url`. Zgodnie z dokumentacją `Pandas`, zmienna `engine`, będzie potrzebna, by możliwe było wyeksportowanie danych na serwer `SQL`.

 W tym miejscu stwórz zmienne `url` oraz `engine`

In [3]:
url = f"postgresql://{username}:{passwd}@{hostname}:{port}/{db_name}"
engine = create_engine(url, echo=False)

 # Załadowanie ramek do obszaru roboczego
 Uzupełnij implementację funkcji `load_raw_data`, która przyjmuje jeden parametr:
 * `file_name` - nazwa pliku do zaczytania
 Jej zadaniem jest wczytanie surowego pliku, zmodyfikowanie nazw kolumn z `NAZWA_KOLUMNY` na `nazwa_kolumny` oraz zwrócenie tak zmodyfikowanej ramki danych.

Mogą się przydać poniższe elementy kursu:
 - `Python-Analiza danych -> Dzień 5 - Pandas -> Obróbka danych - częsć 1`
 - `Python-Analiza danych -> Przygotowanie do zjazdu 3 -> Wstęp do Pandas -> Wczytywanie danych do Pandas` - jakie kodowanie mają pliki?

In [4]:
def load_raw_data(file_name):
    # df = pd.DataFrame
    try:
        df = pd.read_csv(f"../data/raw/{file_name}.csv", index_col=None)
        df.columns = df.columns.str.lower()
        print(f"Utworzono ramkę danych z pliku: {file_name}.csv")
    except FileNotFoundError:
        print(f"Nie ma pliku o nazwie {file_name}.csv")
    except pd.errors.EmptyDataError:
        print(f"Plik {file_name}.csv jest pusty ")
    except pd.errors.ParserError:
        print(f"Nie udało sie załadować pliku {file_name}.csv do Data Frame")
    except Exception as e:
        print(f"Niespodziewany błąd: {e}")

    return df

 # Zaczytanie poszczególnych plików do ramek

 # Eksport danych na bazę
 Zapoznaj się z dokumentacją metody `Pandas` - [to_sql](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html), której zadaniem jest wyeksportowanie ramki na bazę danych.
 Zwróć szczególną uwagę na poniższe parametry:
 * `if_exists` - jak ma się zachować metoda, gdy ładuje dane na bazę,
 * `con` - połączenie do bazy,
 * `name` - nazwa tabeli, do której ramka ma zostać wgrana,
 * `index` - czy dodawać index z ramki jako kolumnę,
 * `chunksize` - maksymalna liczba wierszy wgrywana za jednym razem.

 > **Uwaga:** 
 > Przed eksportem upewnij się, że tabela jest pusta. Zwróć uwagę na pewną subtelną różnicę pomiędzy wyglądem ramki oraz tabeli docelowej na bazie danych.

Następnie uzupełnij implementację metody `export_table_to_db`, która przyjmuje dwa argumenty:
 * `df` - ramka danych do eksportu,
 * `table_name` - nazwa ramki na bazie.

Zalecamy, aby dodać do metody informację, która ramka jest aktualnie ładowana np.:
 `Loading data into {table_name}...`
 > Ze względu na rozmiar ramki `flight_df`, proces ten może potrwać nawet kilkadziesiąt minut! Z tego względu, na potrzeby testów, zalecamy przekazanie do procedury `export_table_to_db` np. pierwszych 5 wierszy, aby sprawdzić, czy działa, a potem wgrać cały zestaw danych - pamiętając o upszednim usunięciu tamtych.

In [6]:
def export_table_to_db(df, table_name):

    try:
        with engine.begin() as conn:
            df.to_sql(
                table_name, conn, if_exists="replace", index_label="id", chunksize=200
            )
            print(f"Załadowano ramkę danych do tabeli {table_name}")
    except Exception as e:
        print("W takich warunkach nie będę pracować: %s" % e)

 ## Wgrywanie danych

In [7]:
def csv_to_db(filename):
    df = load_raw_data(filename)
    if df.empty:
        print("Ramka danych jest pusta! przerwano ładowanie do bazy")
    else:
        export_table_to_db(df, filename)

 ### Wgranie `aircraft_df` do tabeli `aircraft`

In [8]:
csv_to_db("aircraft")

Utworzono ramkę danych z pliku: aircraft.csv
Załadowano ramkę danych do tabeli aircraft


 ### Wgranie `airport_weather_df` do tabeli `airport_weather`

In [9]:
csv_to_db("airport_weather")

Utworzono ramkę danych z pliku: airport_weather.csv
Załadowano ramkę danych do tabeli airport_weather


 ### Wgranie `flight_df` do tabeli `flight`
 > Wykonanie tej komórki będzie czasochłonne za względu na ilość danych w ramce.

In [10]:
csv_to_db("flight")

Utworzono ramkę danych z pliku: flight.csv
Załadowano ramkę danych do tabeli flight


 ### Wgranie `airport_list_df` do tabeli `airport_list`

In [11]:
csv_to_db("airport_list")

Utworzono ramkę danych z pliku: airport_list.csv
Załadowano ramkę danych do tabeli airport_list


 # Sprawdzenie poprawności wykonania notatnika
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [12]:
def test_data_export(table_name, expected_count, expected_schema):
    real_count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", engine).iloc[
        0
    ][0]

    real_schema = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 0", engine)
    real_schema = set(real_schema.columns)

    expected_schema = set(expected_schema)

    diff = real_schema.symmetric_difference(expected_schema)

    assert len(diff) == 0, (
        "Nie zgadzają się kolumny tabel...."
        f"\tOczekiwano: {expected_schema}"
        f"\tOtrzymano: {real_schema}"
        f"\tRóżnica: {diff}"
    )

    assert (
        expected_count == real_count
    ), f"Nie zgadza się liczba wierszy, oczekiwano {expected_count}, otrzymano {real_count}"

 ## Sprawdzenie tabeli `aircraft`
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [13]:
aircraft_expected_count = 7383
aircraft_expected_schema = [
    "id", "manufacture_year", "tail_num", "number_of_seats"]

test_data_export("aircraft", aircraft_expected_count, aircraft_expected_schema)

/tmp/ipykernel_391131/3138711342.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", engine).iloc[


 ## Sprawdzenie tabeli `airport_weather`
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [14]:
airport_weather_expected_count = 46226
airport_weather_expected_schema = [
    "id",
    "station",
    "name",
    "date",
    "awnd",
    "prcp",
    "snow",
    "snwd",
    "tavg",
    "tmax",
    "tmin",
    "wdf2",
    "wdf5",
    "wsf2",
    "wsf5",
    "wt01",
    "wt08",
    "wt02",
    "wt03",
    "wt04",
    "wt09",
    "wt06",
    "wt05",
    "pgtm",
    "wt10",
    "wesd",
    "sn32",
    "sx32",
    "psun",
    "tsun",
    "tobs",
    "wt07",
    "wt11",
    "wt18",
]

test_data_export(
    "airport_weather", airport_weather_expected_count, airport_weather_expected_schema
)

/tmp/ipykernel_391131/3138711342.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", engine).iloc[


 ## Sprawdzenie tabeli `flight`
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [15]:
flight_expected_count = 9251880
flight_expected_schema = [
    "id",
    "month",
    "day_of_month",
    "day_of_week",
    "op_unique_carrier",
    "tail_num",
    "op_carrier_fl_num",
    "origin_airport_id",
    "dest_airport_id",
    "crs_dep_time",
    "dep_time",
    "dep_delay_new",
    "dep_time_blk",
    "crs_arr_time",
    "arr_time",
    "arr_delay_new",
    "arr_time_blk",
    "cancelled",
    "crs_elapsed_time",
    "actual_elapsed_time",
    "distance",
    "distance_group",
    "year",
    "carrier_delay",
    "weather_delay",
    "nas_delay",
    "security_delay",
    "late_aircraft_delay",
]

test_data_export("flight", flight_expected_count, flight_expected_schema)

/tmp/ipykernel_391131/3138711342.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", engine).iloc[


 ## Sprawdzenie tabeli `airport_list`
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [16]:
aircraft_list_expected_count = 97
aircraft_list_expected_schema = [
    "id",
    "origin_airport_id",
    "display_airport_name",
    "origin_city_name",
    "name",
]

test_data_export(
    "airport_list", aircraft_list_expected_count, aircraft_list_expected_schema
)

/tmp/ipykernel_391131/3138711342.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", engine).iloc[


In [17]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.


 # Podsumowanie
 W tym notatniku załadowaliśmy pobrane wcześniej pliki na bazę danych. Dzięki temu stworzyliśmy centralne miejsce ich magazynowania, co wykorzystamy zarówno przy analizie danych, jak i przy późniejszej budowie systemu raportowego.